In [140]:
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import requests
import datetime
import time
import os
import json
import glob
import random
os.chdir("/Users/ujjawalnarayan/Desktop/iphone_delivery_time_tracker")

In [141]:
#declaring base open and colsed docket link
api_link = ""

#declaring chrome header
hdr1 = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

log_file_name_with_path = "resume_helper_logs.txt"

In [142]:
def html_data_grabber(given_page_link,header = hdr1) : 
    """ 
        This function returns dict of information after browsing 
        any given link having soup content of page, status code,
        was it able to grab page data or not with "grabbed data" 
        key and additional helpful message of how url opening went. 

        Parameters: 
            given_page_link    (str)    :  url to browse
                                           ex input : https://www.gao.gov/products/b-419833.2
            header             (dict)   :  dict of browser header information default is set to chrome
        Returns: 
            dict: dict of information after browsing any given link having soup content of page,
            status code,was it able to grab page data or not with "grabbed data" key and
            additional helpful message of how url opening went.
        """
    request_made = requests.get(given_page_link,headers = header)

    #time.sleep(random.randint(3, 8)) #random sleep as per MS guidelines although recommended Crawl-delay on https://www.gao.gov/robots.txt is 10s

    status_code = int(request_made.status_code)
    data = ""
    grabbed_data = False
    if status_code == 200 :
        grabbed_data = True
        data = request_made.text
        message = "Success"
    elif status_code >= 403 :
        message = "banned"
    elif status_code >= 500 :
        message = "Server is down"
    else :
        message = "your internet connection is down"

    return {"content":data, "status_code":status_code, "grabbed_data":grabbed_data, "message": message}

def log_file_presence_checker(link_to_check,log_file_name_with_path) :
    """ 
        This function checks if a links is present
        in log file,it's a helper for implenting crawl resume

        Parameters: 
            link_to_check               (str)  :  single link
            log_file_name_with_path     (str)  :  log filename to check 
        Returns: 
            Boolean : Returns either True or False based on if a given 
                      link is present in log file or not i.e indicating
                      if it is already browsed by crawler or not
        """
    if not os.path.isfile(log_file_name_with_path) :
        return False
    with open(log_file_name_with_path,"r")as f :
        content = f.read()
    if link_to_check in content :
        return True
    else :
        return False
    
def log_file_writer(link_to_write,log_file_name_with_path) :
    """ 
        This function writes links as a log to given file.
        helps in resuming crawl

        Parameters: 
            link_to_write               (str/list)  :  single link or list of links
            log_file_name_with_path     (str)       :  log filename to write 
        Returns: 
            null
        """
    with open(log_file_name_with_path,"a") as f :
        if isinstance(link_to_write, list) :
            for single_link in link_to_write :
                f.write(str(single_link) + "\n")
        else :
            f.write(str(link_to_write) + "\n")
            
def temp_file_writer(all_data,temp_file_with_path) :
    """ 
        This function writes current crawl data which are list of dict docket data
        provided by "temp_open_close_all_docket_data_downloader" function.
        This is a helper function and benefits in resuming crawl during interruptions.

        Parameters: 
            all_data                (list) :  list of dicts docket data
            temp_file_with_path     (str)  :  temp json store filename with path 
        Returns: 
            null
        """
    with open(temp_file_with_path,"a") as f :
        if isinstance(all_data, list) :
            for single_data in all_data :
                f.write(json.dumps(single_data) + "\n")
        else :
            f.write(json.dumps(all_data) + "\n")

In [143]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                if k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    if len(values) == 0 :
        values = [""]
    return values

def grab_data_from_api(iphone_product_code,zip_code,carrier,check_in_log=False) :
    api_link = 'https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode={zip_code}&parts.0={iphone_product_code}&cppart={carrier}'.format(iphone_product_code=iphone_product_code,zip_code=zip_code,carrier=carrier)
    if check_in_log :
        if log_file_presence_checker(api_link,log_file_name_with_path) :
            return {}
        
    print(api_link)
    base_link_content = html_data_grabber(api_link)
    if base_link_content["grabbed_data"] :
        content = json.loads(base_link_content["content"])
    else :
        print(api_link,base_link_content["message"],base_link_content["status_code"])
        raise
    return_dict = {}
    return_dict["crawl_time"] = datetime.datetime.today().strftime("%d-%m-%Y")
    return_dict["api_link"] = api_link
    return_dict["product_name"] = json_extract(content, "storePickupProductTitle")[0]
    return_dict["zip_code"] = zip_code
    #return_dict["Carrier"] = carrier
    return_dict["all_delivery_message"] = "|".join(json_extract(json_extract(content, "deliveryOptionMessages"),"displayName"))
    all_delivery_dates =  json_extract(content, "encodedUpperDateStringForID") + json_extract(content, "encodedUpperDateString") + json_extract(content, "encodedUpperDateStringForIDL")
    all_delivery_dates = sorted([datetime.datetime.strptime(i, '%Y%m%d') for i in all_delivery_dates if i != ""])

    if len(all_delivery_dates) > 2 :
        print("ALERTTTTTTTT!!!!!!!!!!!!!!!!!!!")
    if len(all_delivery_dates) != 0 :
        return_dict["quickest delivery Date most probably from store"] = all_delivery_dates[0].strftime("%Y-%m-%d")
        return_dict["last delivery Date"] = all_delivery_dates[-1].strftime("%Y-%m-%d")
        return_dict["Ships/Delivers"] = "Delivers"

    if check_in_log : 
        log_file_writer(api_link,log_file_name_with_path)
        
    return return_dict

In [144]:
all_zip_codes = ["10128","90036","75205","30326","60611","98105"]

In [145]:
#driver

In [147]:
file_count = 0
df = pd.read_csv("product_code.csv")
all_search_pages_results = len(df) * [""]
for i in range(len(df)) :
    current_product = df.loc[i].to_dict()
    for single_zip in all_zip_codes :
        curr_carrier = current_product["carrier_code"]
        data_from_api = grab_data_from_api(current_product["product_code"],single_zip,curr_carrier,True)
        if len(data_from_api) != 0 :
            final_data = {**current_product, **data_from_api}
            temp_file_writer(final_data,"apple_temp_file.json")
            print(data_from_api["product_name"],curr_carrier)
            time.sleep(random.randint(3, 5))
    file_count = file_count + 1
    percent_calci = (float(file_count)/float(len(all_search_pages_results)))*100.0
    print ("working on link no " + str(file_count) + " of total links " + str(len(all_search_pages_results)) + " " + str(percent_calci) + " % completed")

        
        

working on link no 1 of total links 478 0.20920502092050208 % completed
https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=60611&parts.0=MLHT3LL/A&cppart=ATT_IPHONE13
ALERTTTTTTTT!!!!!!!!!!!!!!!!!!!
iPhone 13 mini 256GB Midnight ATT_IPHONE13
https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=98105&parts.0=MLHT3LL/A&cppart=ATT_IPHONE13
iPhone 13 mini 256GB Midnight ATT_IPHONE13
working on link no 2 of total links 478 0.41841004184100417 % completed
https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=10128&parts.0=MLHY3LL/A&cppart=ATT_IPHONE13
iPhone 13 mini 512GB Midnight ATT_IPHONE13
https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=90036&parts.0=MLHY3LL/A&cppart=ATT_IPHONE13
iPhone 13 mini 512GB Midnight ATT_IPHONE13
https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=75205&parts.0=MLHY3LL/A&cppart=ATT_IPHONE

In [148]:
import pandas as pd
import json
import os
os.chdir("/Users/ujjawalnarayan/Desktop/iphone_delivery_time_tracker")
log_file_name_with_path = "apple_temp_file.json"
with open(log_file_name_with_path,"r") as f :
    all_data_to_be_written = [json.loads(i) for i in f.readlines() if json.loads(i) != False]
df = pd.DataFrame(all_data_to_be_written)

In [149]:
df

,Capacity(GB),Finish,product_code,Model,carrier_code,crawl_time,api_link,product_name,zip_code,all_delivery_message,quickest delivery Date most probably from store,last delivery Date,Ships/Delivers
0,128GB,Midnight,MLHM3LL/A,iphone13 mini,ATT_IPHONE13,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 13 mini 128GB Midnight,10128,Today from Store 9:30a.m. to 11:30a.m. — $9.00...,2021-12-20,2021-12-21,Delivers
1,128GB,Midnight,MLHM3LL/A,iphone13 mini,ATT_IPHONE13,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 13 mini 128GB Midnight,90036,"Tomorrow 10:30a.m. - 12:30p.m. — $9.00|Tue, De...",2021-12-20,2021-12-22,Delivers
2,128GB,Midnight,MLHM3LL/A,iphone13 mini,ATT_IPHONE13,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 13 mini 128GB Midnight,75205,Today from Store 9:30a.m. to 11:30a.m. — $9.00...,2021-12-20,2021-12-21,Delivers
3,128GB,Midnight,MLHM3LL/A,iphone13 mini,ATT_IPHONE13,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 13 mini 128GB Midnight,30326,Today from Store 11a.m. to 1p.m. — $9.00|Tomor...,2021-12-20,2021-12-22,Delivers
4,128GB,Midnight,MLHM3LL/A,iphone13 mini,ATT_IPHONE13,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 13 mini 128GB Midnight,60611,Today from Store 11a.m. to 1p.m. — $9.00|Tomor...,2021-12-20,2021-12-21,Delivers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,NaN,NaN,MHD33LL/A,iphone 11,UNLOCKED/US,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 11 128GB Green,90036,Today from Store 10:30a.m. to 12:30p.m. — $9.0...,2021-12-20,2022-01-05,Delivers
2864,NaN,NaN,MHD33LL/A,iphone 11,UNLOCKED/US,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 11 128GB Green,75205,Dec 29 - Jan 5 — Free,2022-01-05,2022-01-05,Delivers
2865,NaN,NaN,MHD33LL/A,iphone 11,UNLOCKED/US,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 11 128GB Green,30326,Dec 29 - Jan 5 — Free,2022-01-05,2022-01-05,Delivers
2866,NaN,NaN,MHD33LL/A,iphone 11,UNLOCKED/US,20-12-2021,https://www.apple.com/shop/fulfillment-message...,iPhone 11 128GB Green,60611,Dec 29 - Jan 5 — Free,2022-01-05,2022-01-05,Delivers


In [155]:
df["all_delivery_message"].tolist()#.str.split("|",expand = True)

['Today from Store 9:30a.m. to 11:30a.m. — $9.00|Tomorrow — Free',
 'Tomorrow 10:30a.m. - 12:30p.m. — $9.00|Tue, Dec 21 — Free|Delivery with Setup — Free',
 'Today from Store 9:30a.m. to 11:30a.m. — $9.00|Tomorrow — Free|Delivery with Setup — Free',
 'Today from Store 11a.m. to 1p.m. — $9.00|Tomorrow — Free|Delivery with Setup — Free',
 'Today from Store 11a.m. to 1p.m. — $9.00|Tomorrow — Free|Delivery with Setup — Free',
 'Tue, Dec 21 — Free|Delivery with Setup — Free',
 'Today from Store 10a.m. to 12p.m. — $9.00|Tomorrow — Free',
 'Tomorrow 10:30a.m. - 12:30p.m. — $9.00|Tue, Dec 21 — Free|Delivery with Setup — Free',
 'Tomorrow — Free|Delivery with Setup — Free',
 'Today from Store 11a.m. to 1p.m. — $9.00|Tomorrow — Free|Delivery with Setup — Free',
 'Today from Store 11a.m. to 1p.m. — $9.00|Tomorrow — Free|Delivery with Setup — Free',
 'Tue, Dec 21 — Free|Delivery with Setup — Free',
 'Today from Store 9:30a.m. to 11:30a.m. — $9.00|Tomorrow — Free',
 'Tomorrow 10:30a.m. - 12:30p.m. 

In [111]:
grab_data_from_api("MGLU3LL/A","10128","UNLOCKED/US")

https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=10128&parts.0=MGLU3LL/A&cppart=UNLOCKED/US
[]


{'crawl_time': '17-12-2021',
 'api_link': 'https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=10128&parts.0=MGLU3LL/A&cppart=UNLOCKED/US',
 'product_name': '',
 'zip_code': '10128'}

In [100]:
ll = html_data_grabber("https://www.apple.com/shop/fulfillment-messages?geoLocated=false&mt=regular&postalCode=10128&parts.0=MLMU3LL/A&cppart=TMOBILE_IPHONE13")
ll = json.loads(ll["content"])

In [101]:
ll

{'head': {'status': '200', 'data': {}},
 'body': {'content': {'pickupMessage': {'stores': [{'storeEmail': 'uppereastside@apple.com',
      'storeName': 'Upper East Side',
      'reservationUrl': 'http://www.apple.com/retail/uppereastside/',
      'makeReservationUrl': 'http://www.apple.com/retail/uppereastside/',
      'state': 'NY',
      'storeImageUrl': 'https://rtlimages.apple.com/cmc/dieter/store/4_3/R582.png?resize=828:*&output-format=jpg',
      'country': 'US',
      'city': 'New York',
      'storeNumber': 'R582',
      'partsAvailability': {'MLMU3LL/A': {'storePickEligible': True,
        'storeSearchEnabled': True,
        'storeSelectionEnabled': True,
        'storePickupQuote': 'Today at Apple Upper East Side',
        'storePickupQuote2_0': '<span class="as-pickup-quote-availability-quote">Today</span> at <button type="button" class="rf-pickup-quote-overlay-trigger as-retailavailabilitytrigger-infobutton retail-availability-search-trigger as-buttonlink" data-ase-overlay=

In [110]:
"|".join(json_extract(json_extract(ll, "deliveryOptionMessages"),"displayName"))

'Today from Store 9:30a.m. to 11:30a.m. — $9.00|Tue, Dec 21 — Free'

In [ ]:
TMOBILE_IPHONE13

ATT_IPHONE13







ATT_IPHONE13PRO

SPRINT_IPHONE13PRO

TMOBILE_IPHONE13PRO


VERIZON_IPHONE13PRO

UNLOCKED/US

In [113]:
mm = {"@context":"https://schema.org","@type":"Product","name":"iPhone 11","url":"https://www.apple.com/shop/buy-iphone/iphone-11","mainEntityOfPage":"https://www.apple.com/shop/buy-iphone/iphone-11","offers":[{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH903LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHC73LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH903LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH983LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8F3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHC83LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH8Q3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH8N3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHCR3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8D3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8H3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCY3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCG3LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHD13LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCF3LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH933LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHC43LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH933LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH993LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8Y3LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH8K3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHCP3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH943LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH963LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHCQ3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH953LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHCW3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHC93LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHD23LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8Y3LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHD33LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCE3LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH913LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHC63LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8E3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH973LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH983LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH9A3LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH913LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCD3LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHC53LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH8L3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8G3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH953LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH8P3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHD03LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH943LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCC3LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH963LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH993LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH973LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCA3LL/A+verizon"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH8J3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH8M3LL/A+att"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH923LL/A+sprint_US"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MHCX3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHCV3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MHCU3LL/A+unlocked"},{"@type":"Offer","priceCurrency":"USD","price":549.00,"sku":"MH9A3LL/A+tmobile"},{"@type":"Offer","priceCurrency":"USD","price":499.00,"sku":"MH923LL/A+tmobile"}],"image":"https://store.storeimages.cdn-apple.com/4982/as-images.apple.com/is/iphone-11-og-201909?wid=600&hei=315&fmt=jpeg&qlt=95&.v=1566949840644"}

In [123]:
lk = sorted([i["sku"] for i in mm["offers"] if "sprint" not in i["sku"]])

In [126]:
[print(",".join(i.split("+"))) for i in lk]

MH8D3LL/A,att
MH8E3LL/A,att
MH8F3LL/A,att
MH8G3LL/A,att
MH8H3LL/A,att
MH8J3LL/A,att
MH8K3LL/A,att
MH8L3LL/A,att
MH8M3LL/A,att
MH8N3LL/A,att
MH8P3LL/A,att
MH8Q3LL/A,att
MH8Y3LL/A,tmobile
MH903LL/A,tmobile
MH913LL/A,tmobile
MH923LL/A,tmobile
MH933LL/A,tmobile
MH943LL/A,tmobile
MH953LL/A,tmobile
MH963LL/A,tmobile
MH973LL/A,tmobile
MH983LL/A,tmobile
MH993LL/A,tmobile
MH9A3LL/A,tmobile
MHC43LL/A,verizon
MHC53LL/A,verizon
MHC63LL/A,verizon
MHC73LL/A,verizon
MHC83LL/A,verizon
MHC93LL/A,verizon
MHCA3LL/A,verizon
MHCC3LL/A,verizon
MHCD3LL/A,verizon
MHCE3LL/A,verizon
MHCF3LL/A,verizon
MHCG3LL/A,verizon
MHCP3LL/A,unlocked
MHCQ3LL/A,unlocked
MHCR3LL/A,unlocked
MHCU3LL/A,unlocked
MHCV3LL/A,unlocked
MHCW3LL/A,unlocked
MHCX3LL/A,unlocked
MHCY3LL/A,unlocked
MHD03LL/A,unlocked
MHD13LL/A,unlocked
MHD23LL/A,unlocked
MHD33LL/A,unlocked


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]